In [1]:
import sys
sys.path.append('../Data')

import pandas as pd
import numpy as np
import time
from CommentDataset import CommentDataset

In [2]:
comment_list_name = "AIhyouka_sannkou_gyakutaimiokuri"
comment_list_path = "2023_07_11/" + comment_list_name + ".csv"

In [ ]:
# 指定すべき文字コードはファイルによる

df = pd.read_csv("../Data/yahoo/"+comment_list_path, encoding="utf-8", header=None)
# comment_list = df[0][1:].to_numpy().tolist()
comment_list = df[0][1:10].to_numpy().tolist()
comment_dataset = CommentDataset(comment_list)
comment_dataset.delete_comment_specified_string(50)
comment_dataset.formatted_input_hlda(appear_tagging_list=["名詞", "形容詞", "動詞"], stop_tagging_list=["助詞", "助動詞", "非自立"])

In [4]:
from hlda.sampler import HierarchicalLDA

#default
# n_samples = 500       # no of iterations for the sampler
n_samples = 50       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 50   # the number of iterations between printing a brief summary of the topics so far
n_words = 10           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [5]:
hlda = HierarchicalLDA(comment_dataset.comment_list, comment_dataset.voc, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling

.................................................. 50
topic=0 level=0 (documents=1200): 思う, 判断, れる, データ, 子供, 職員, 虐待, それ, 仕事, 問題, 
    topic=1 level=1 (documents=481): 保護, 判断, 人間, 責任, 使う, 対応, できる, 言う, 数値, 思う, 
        topic=2 level=2 (documents=18): 解体, 委員, 怪しい, 人事, 組織, 手段, クビ, 強化, 不幸, 失態, 
        topic=7 level=2 (documents=51): 人間, 判断, 機械, 最終, 技術, 進む, 逃れ, 委ねる, 公表, 便利, 
        topic=9 level=2 (documents=10): 警察, 時間, 部署, 連携, 新しい, 頼み, 役割, 今度, 仕組み, 守る, 
        topic=10 level=2 (documents=24): 子供, 日本, アメリカ, 対策, 疑い, 予算, 守る, あなた, 施設, 国民, 
        topic=14 level=2 (documents=85): 長期, 欠席, 保育園, 問題, その後, 情報, 面会, 対応, 得る, 女児, 
        topic=16 level=2 (documents=6): パンク, 馬鹿, 受け止める, 疑わしい, つながる, 若干, エンド, 側面, 困惑, 恐れる, 
        topic=24 level=2 (documents=11): 組織, 関係, 強い, センター, セン, 支援, 下部, １つ, 使命, 集団, 
        topic=27 level=2 (documents=17): 人類, 滅ぼす, 業務, 定義, 生命, 不用意, みる, 骨頂, 全て, 飼い主, 
        topic=29 level=2 (documents=15): 犯罪, 叩く, 母親, 起きる, 権限, 産む, 文句, 防げる, こども, つける, 
   

In [6]:
from model.HldaModel import ExpandHldaModel
import os

create_date = "2023_07_26"
parameter_name = "default"
model_name = comment_list_name + "_" + parameter_name + ".pickle"
model_dir = "model/pickle/" + create_date + "/" +model_name
create_date_path = "model/pickle/" + create_date

if not os.path.exists(create_date_path):
    os.makedirs(create_date_path)

expandHlda = ExpandHldaModel(pickle_path=hlda)
expandHlda.save_zipped_pickle(filename=model_dir)

In [7]:
expandHlda.get_topic_multi_document(comment_list=comment_list, corpus=comment_dataset.comment_list)

AttributeError: 'list' object has no attribute 'replace'